In [ ]:
import pandas as pd
import sklearn

In [ ]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")

In [ ]:
df.shape

In [ ]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df.duration.describe()

In [ ]:
df.head()

In [ ]:
df_filtered = df[(df.duration >= 1) & (df.duration <= 60)]
(df.shape[0] - df_filtered.shape[0]) / df.shape[0]
df_train = df[["duration", "PULocationID", "DOLocationID"]]
df_train[["PULocationID", "DOLocationID"]] = df_train[["PULocationID", "DOLocationID"]].astype(str)

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer()

In [ ]:
train_dict = df_train.to_dict(orient="records")

In [ ]:
X_train = dv.fit_transform(train_dict)

In [ ]:
len(dv.feature_names_)

In [ ]:
y_train = df_train["duration"].values

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
train_predictions = lr.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_train, train_predictions, squared=False)

In [ ]:
df_validation = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

In [ ]:
df_validation.tpep_dropoff_datetime = pd.to_datetime(df_validation.tpep_dropoff_datetime)
df_validation.tpep_pickup_datetime = pd.to_datetime(df_validation.tpep_pickup_datetime)
df_validation["duration"] = df_validation.tpep_dropoff_datetime - df_validation.tpep_pickup_datetime
df_validation.duration = df_validation.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df_validation_filtered = df_validation[(df_validation.duration >= 1) & (df_validation.duration <= 60)]
df_validation_filtered = df_validation_filtered[["duration", "PULocationID", "DOLocationID"]]
df_validation[["PULocationID", "DOLocationID"]] = df_validation[["PULocationID", "DOLocationID"]].astype(str)

In [ ]:
X_valid = dv.fit_transform(df_validation.to_dict(orient="records"))

In [ ]:
y_valid = df_validation["duration"].values

In [ ]:
valid_predict = lr.predict(X_valid)

In [ ]:
mean_squared_error(y_valid, valid_predict, squared=False)